In [ ]:
import pandas as pd
import torch
import os
import math
import numpy as np
import matplotlib.pyplot as plt
from shapely.wkt import loads
import torch_geometric
import random 


### Turning Regular, Tabular Data into Graph-Structured Data
Our data is initially formatted as a table of emissions readings

In [ ]:
coal_mining = pd.read_csv('data/fossil-fuels/asset_coal-mining_emissions.csv')
coal_mining['coords'] = coal_mining['st_astext'].apply(lambda x: loads(x).coords[0])
coal_mining.head(15)

Each emission site has 5 readings - one for each gas type (co2, ch4, n2o, co2e_100yr, co2e_20yr)

In [ ]:
coal_mining.asset_id.value_counts().unique() # There are exactly 5 occurences of every ID - no more no less

We can filter the dataframe to just one of these to draw our edges, since each of the 5 readings represents the same site in the same location, just a different gas. Before we do that, we can map the IDs from a (m-n) range to (0-k) where k = # unique asset IDs - 1.

In [ ]:
def generate_id_mapping(ids):
    '''
    Arg(s):
        ids - numpy.ndarray of UNIQUE asset_ids of unknown range and length
    Output:
        id_mapping - dictionary where
            key: original asset_id from df (e.g. 136113483)
            val: ID in range 0-k, where k = # of unique IDs - 1.
    '''
    id_mapping = {}
    for i, asset in enumerate(ids):
        id_mapping[asset] = i
    
    return id_mapping

In [ ]:
unique_ids = coal_mining.asset_id.unique()
id_mapping = generate_id_mapping(unique_ids)

coal_mining['asset_id'] = coal_mining['asset_id'].apply(lambda x: id_mapping[x])
coal_mining

Now we will filter to only one gas

In [ ]:
coal_ch4_emissions = coal_mining[coal_mining['gas'] == 'ch4']
coal_ch4_emissions

Now we can start drawing edges using Haversine distance. First, we pull out id-coord pairs - like (id, (lat,lon))

In [ ]:
id_coord_pairs = list(zip(coal_ch4_emissions['asset_id'], coal_ch4_emissions['coords']))
id_coord_pairs

Then we can establish our distance metric - Haversine

In [ ]:
def haversine(pt1, pt2):
    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(pt1[1])
    lon1 = math.radians(pt1[0])
    lat2 = math.radians(pt2[1])
    lon2 = math.radians(pt2[0])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = 6371.0 * c  # Radius of the Earth in kilometers (you can use 3959.0 for miles)

    return distance

In [ ]:
def connect_nodes(id_coord_pairs, distance_threshold):
    '''
    Arg(s):
        id_coord_pairs: A list of tuples containing
            - a normalized asset ID, e.g. 4
            - a tuple lat-lon pair, e.g. (100, 30)
    Output:
        edge_index: A 2xN tensor representing every edge. Think of the first row 
        as all the starting points of each edge, and the second row as the finish point
    '''
    start_of_edges = []
    end_of_edges = []

    for pair_a in id_coord_pairs:
        id_a = pair_a[0]
        coords_a = pair_a[1]
        for pair_b in id_coord_pairs:
            id_b = pair_b[0]
            coords_b = pair_b[1]
            if id_a != id_b:
                distance = haversine(coords_a, coords_b)
                if distance < distance_threshold:
                    start_of_edges.append(id_a)
                    end_of_edges.append(id_b)

    edge_index = [start_of_edges, end_of_edges]
    return torch.tensor(edge_index)

Here I will aggregate all of this into a function that can take in a csv of data and save a bunch of corresponding edge indices based on distances.


In [ ]:
def csv_to_data_and_edges(df, distances, object_folder_path, gas=None):
    '''
    Inputs:
        df (pd.Dataframe)
            -processed dataframe (i.e. number of rows finalized, time granularities already squashed)
        gas (str)
            -options are co2, ch4, n2o, co2e_100yr, co2e_20yr
            -check the data CSV first before picking one!
        distances (arr[int])
            -array of desired distance thresholds
        object_folder_path (str)
            -path of folder to store the resultant pyg objects
            
    '''

    # First load in the CSV and convert coords to numerical values
    df['coords'] = df['st_astext'].apply(lambda x: loads(x).coords[0])

    # Generate ID mapping for PyG convention and remap df IDs
    unique_ids = df.asset_id.unique()
    id_mapping = generate_id_mapping(unique_ids)
    df['asset_id'] = df['asset_id'].apply(lambda x: id_mapping[x])

    # Filter by desired gas if needed
    if gas != None:
        df = df[df['gas'] == gas]

    # Generate pairs of form (ID #, (lat, lon))
    id_coord_pairs = list(zip(df['asset_id'], df['coords']))
    
    # We will reformat and save these pairs for visualizations later!
    node_coords = {}
    for pair in id_coord_pairs:
        node_coords[pair[0]] = pair[1]
    torch.save(node_coords, object_folder_path + 'location-mapping')

    # Make edge indices and save them locally
    for distance in distances:
        edge_index = connect_nodes(id_coord_pairs, distance)
        path = object_folder_path + str(distance) + 'km'
        torch.save(edge_index, path)

    return df
    

In [ ]:
# Get value counts of the 'Category' column
value_counts = coal_ch4_emissions['asset_type'].value_counts()

# Plot the bar chart
value_counts.plot(kind='bar', color='skyblue', edgecolor='black')

# Set plot labels and title
plt.xlabel('Categories')
plt.ylabel('Count')
plt.title('Bar Chart of Category Counts')

# Show the plot
plt.show()

### Oil and Gas Preprocessing

First we will filter the dataset to a relevant subset.

In [ ]:
oil_gas_df = pd.read_csv('data/fossil-fuels/asset_oil-and-gas-production-and-transport_emissions.csv')
oil_gas_df['start_time'] = pd.to_datetime(oil_gas_df['start_time'])
oil_gas_df['year'] = oil_gas_df['start_time'].dt.year
oil_gas_df = oil_gas_df[  (oil_gas_df['year'] == 2021)
                        & ((oil_gas_df['gas'] == 'co2') | (oil_gas_df['gas'] == 'ch4'))]
oil_gas_df.head()

The oil and gas dataset contains emission quantities for two different gases (ch4 and co2) - we can use these both as features for classification and as labels for regression.

In [ ]:
ch4_co2_vals = oil_gas_df.pivot(index='asset_id', columns='gas', values='emissions_quantity').reset_index()
ch4_co2_vals

In [ ]:
oil_gas_ch4_co2_df = pd.merge(oil_gas_df,
                     ch4_co2_vals,
                     on=['asset_id'])
oil_gas_ch4_co2_df = oil_gas_ch4_co2_df.drop_duplicates(subset='asset_id') # we now have redundant duplicates since each row stores ch4 and co2 emission
oil_gas_ch4_co2_df.head() # may need to scroll to the right to see ch4 and co2 stored!

In [ ]:
distances = [1,2,3,4,5,25,50,100,200,500,1000,2000,5000,10000]
oil_gas_ch4_co2_df = csv_to_data_and_edges(df=oil_gas_ch4_co2_df, distances=distances, object_folder_path='pyg_objects/oil+gas/co2+ch4-', gas=None)

In [ ]:
# Get value counts of the 'Category' column
value_counts = oil_gas_ch4_co2_df['asset_type'].value_counts()

# Plot the bar chart
value_counts.plot(kind='bar', color='skyblue', edgecolor='black')

# Set plot labels and title
plt.xlabel('Categories')
plt.ylabel('Count')
plt.title('Bar Chart of Category Counts')

# Show the plot
plt.show()

Now we can turn this dataframe into a PyG `Data` object:

In [ ]:
from torch_geometric.data import Data
from torch_geometric.transforms import NormalizeFeatures, Compose
import torch.nn.functional as F
import torch_geometric.transforms as T

def cleaned_df_to_data(df, task, gases=None):
    '''
    Inputs:
        df: 
            -the dataframe you have minus columns you do not intend to use in the Data object
        task: 
            -'regression' or 'classification'
        gases (optional): 
            -a list of gas names (str) if the data has multiple types of emission
            -requires each emission to be stored in a column bearing the gas's name (`ch4`, `co2`, etc.) 
    Outputs:
        data: a PyG `Data` object (with no edge index! added dynamically in testing)
    '''

    # We convert country of location to a categorical feature.
    # The labels are adapted depending on the task.
    if task == 'classification':
        labels = torch.tensor(pd.factorize(df['asset_type'])[0])
        df = df.drop(['asset_type'], axis=1)
        df_cat = df[['iso3_country']]
        dummies = pd.get_dummies(df_cat)
        df = df.drop(['iso3_country'], axis=1)
        df = pd.concat([df, dummies], axis=1)
    else:
        if gases != None:
            labels = torch.tensor(df[gases].values)
        else:
            labels = torch.tensor(df['emissions_quantity'].values)
        df_cat = df[['iso3_country', 'asset_type']]
        dummies = pd.get_dummies(df_cat)
        df = df.drop(['iso3_country', 'asset_type'], axis=1)
        df = pd.concat([df, dummies], axis=1)
    
    df = df.astype(float)
    features = torch.tensor(df.values)

    permuted_indices = torch.randperm(labels.size(0))
    labels = labels[permuted_indices]
    features = features[permuted_indices]

    data = Data(x=features, y=labels)

    # Train and test masks
    train_len = round(len(data.y) * 0.8) # 80% of total data will be for training 
    test_len = len(data.y) - train_len # the rest (20%) for testing

    train_mask = ([True] * train_len) + ([False] * test_len) 
    test_mask = ([False] * train_len) + ([True] * test_len) 

    data.train_mask = torch.tensor(train_mask)
    data.test_mask = torch.tensor(test_mask)

    transform = T.NormalizeFeatures()
    data = transform(data)

    return data
    

In [ ]:
oil_gas_ch4_co2_df = oil_gas_ch4_co2_df.drop(['asset_id', 'original_inventory_sector', 'start_time', 'end_time', 
                     'temporal_granularity', 'gas', 'emissions_quantity', 'emissions_factor_units', 'created_date', 
                     'modified_date', 'asset_name', 'st_astext', 'coords', 'year'], axis=1)
oil_gas_ch4_co2_df

In [ ]:
oil_gas_classification_data = cleaned_df_to_data(oil_gas_ch4_co2_df, 'classification', ['ch4','co2'])
oil_gas_regression_data = cleaned_df_to_data(oil_gas_ch4_co2_df, 'regression', ['ch4','co2'])

oil_gas_classification_data, oil_gas_regression_data

In [ ]:
torch.save(oil_gas_classification_data, 'pyg_objects/oil+gas/classification_data')
torch.save(oil_gas_regression_data, 'pyg_objects/oil+gas/regression_data')

### Coal Mining Data Prep

In [ ]:
distances = [1,2,3,4,5,25,50,100,200,500,1000,2000,5000,10000]
coal_ch4_emissions = csv_to_data_and_edges(df=coal_ch4_emissions, distances=distances, object_folder_path='pyg_objects/coal/coal-ch4-edges-', gas=None)

In [ ]:
coal_ch4_emissions = coal_ch4_emissions.drop(['asset_id', 'original_inventory_sector', 'start_time', 'end_time', 
                     'temporal_granularity', 'gas', 'emissions_factor_units',  'capacity_units', 'capacity', 'capacity_factor', 
                     'activity_units', 'created_date', 'modified_date', 'asset_name', 'st_astext', 'coords', 'lat_lon'], axis=1)



In [ ]:
coal_classification_data = cleaned_df_to_data(coal_ch4_emissions, 'classification')
coal_regression_data = cleaned_df_to_data(coal_ch4_emissions, 'regression')

torch.save(coal_classification_data, 'pyg_objects/coal/coal_classification_data')
torch.save(coal_regression_data, 'pyg_objects/coal/coal_regression_data')

### Statistics on Each Dataset

Beginning with coal, we will see how many edges are present at each distance threshold:

In [29]:
distances = [1,2,3,4,5,25,50,100,200,500,1000,2000,5000,10000]
print("COAL DATASET")
for d in distances:
    edges = torch.load('pyg_objects/coal/coal-ch4-edges-' + str(d) + 'km')
    print(d, "km threshold:", edges.shape[-1], "edges")

print("OIL/GAS DATASET")
for d in distances:
    edges = torch.load('pyg_objects/oil+gas/co2+ch4-' + str(d) + 'km')
    print(d, "km threshold:", edges.shape[-1], "edges")

COAL DATASET
1 km threshold: 452 edges
2 km threshold: 968 edges
3 km threshold: 1788 edges
4 km threshold: 2772 edges
5 km threshold: 3930 edges
25 km threshold: 33032 edges
50 km threshold: 74210 edges
100 km threshold: 160670 edges
200 km threshold: 348012 edges
500 km threshold: 942714 edges
1000 km threshold: 1348632 edges
2000 km threshold: 1796560 edges
5000 km threshold: 3927088 edges
10000 km threshold: 5349564 edges
OIL/GAS DATASET
1 km threshold: 10 edges
2 km threshold: 12 edges
3 km threshold: 12 edges
4 km threshold: 18 edges
5 km threshold: 20 edges
25 km threshold: 228 edges
50 km threshold: 596 edges
100 km threshold: 1446 edges
200 km threshold: 3028 edges
500 km threshold: 7192 edges
1000 km threshold: 12312 edges
2000 km threshold: 26152 edges
5000 km threshold: 90260 edges
10000 km threshold: 209010 edges


In [30]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [31]:
import pkg_resources
import subprocess

# Get a list of installed packages
installed_packages = pkg_resources.working_set
installed_packages_names = [pkg.project_name for pkg in installed_packages]

# Get a list of imported packages in the notebook
imported_packages = set()
for line in open('node-connection.ipynb', 'r').readlines():
    if 'import ' in line or 'from ' in line:
        parts = line.split()
        if len(parts) > 1:
            imported_packages.add(parts[1].split('.')[0])

# Filter the installed packages based on the imported packages
filtered_packages = [package for package in installed_packages_names if package.lower() in imported_packages]

# Generate the requirements.txt file
with open('requirements.txt', 'w') as f:
    for package in filtered_packages:
        f.write(f"{package}\n")
